In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from arena import Arena, Aperture, Mouse, visualize_arena
from tqdm import tqdm
import os
from scipy.interpolate import interp1d
import functools

BASE_PATH = "/Users/yc/Desktop"

# Optimized get_visibility function using NumPy operations more efficiently
def get_visibility(source, target, aperture):
    """Optimized visibility check using direct vector operations"""
    # Convert tuples to numpy arrays
    source_array = np.array(source)
    target_array = np.array(target)
    left_wall = np.array([aperture.left_wall_edge[0], aperture.left_wall_edge[1], target[2]])
    right_wall = np.array([aperture.right_wall_edge[0], aperture.right_wall_edge[1], target[2]])
    
    # Vector calculations with direct NumPy operations
    v_t2Lwall = left_wall - target_array
    v_t2Rwall = right_wall - target_array
    v_t2s = source_array - target_array
    
    # Normalize only once where needed
    norm_L = np.linalg.norm(v_t2Lwall[:2])
    norm_R = np.linalg.norm(v_t2Rwall[:2])
    norm_S = np.linalg.norm(v_t2s[:2])
    
    if norm_L > 0 and norm_R > 0 and norm_S > 0:
        v_t2Lwall_norm = v_t2Lwall[:2] / norm_L
        v_t2Rwall_norm = v_t2Rwall[:2] / norm_R
        v_t2s_norm = v_t2s[:2] / norm_S
        
        # Direct 2D cross product calculation (faster than np.cross for 2D)
        cross_LS = v_t2Lwall_norm[0] * v_t2s_norm[1] - v_t2Lwall_norm[1] * v_t2s_norm[0]
        cross_RS = v_t2s_norm[0] * v_t2Rwall_norm[1] - v_t2s_norm[1] * v_t2Rwall_norm[0]
        
        return cross_LS >= 0 and cross_RS >= 0
    return False

# Cache for circle points to avoid recalculation
circle_points_cache = {}

# Use functools.lru_cache to avoid redundant calculations
@functools.lru_cache(maxsize=1024)
def get_circle_points(circle_center_str, radius, d_theta):
    """Calculate circle perimeter points (cached)"""
    # Convert string back to tuple
    parts = circle_center_str.split(',')
    circle_center = (float(parts[0]), float(parts[1]), float(parts[2]))
    
    angles = np.arange(0, 2 * np.pi, d_theta)
    circle_points = []
    
    for angle in angles:
        x_circle = circle_center[0] + radius * np.cos(angle)
        z_circle = circle_center[2] + radius * np.sin(angle)
        y_circle = circle_center[1]  # Y remains constant
        circle_points.append((x_circle, y_circle, z_circle, angle))
    
    return circle_points

def get_visible_angles(source, circle_center, radius, aperture):
    """Calculate visible angles with caching"""
    d_theta = np.pi/180  # 1-degree increments
    
    # Convert circle_center to string for caching
    circle_center_str = f"{circle_center[0]},{circle_center[1]},{circle_center[2]}"
    
    # Get cached circle points
    circle_points = get_circle_points(circle_center_str, radius, d_theta)
    visible_angles = []
    
    for point in circle_points:
        if get_visibility(source, point[:3], aperture):
            visible_angles.append(point[3])  # The angle
    
    return visible_angles

# Cache for segment area calculations
@functools.lru_cache(maxsize=1024)
def get_segment_area_key(angles_key, radius):
    """Calculate segment area with caching"""
    # Convert string back to angles array
    angles = np.fromstring(angles_key, sep=',')
    
    if len(angles) == 0:
        return np.array([0.0, 0.0])
    
    angles = np.sort(angles)
    
    if 0 in angles:
        diffs = np.diff(angles, append=angles[-1]-angles[0])
    else:
        diffs = np.abs(np.diff(angles, append=angles[0]))
    
    central_angle = np.max([np.max(diffs), np.min(diffs)])
    area = 0.5 * radius**2 * (central_angle - np.sin(central_angle))
    area_c = np.abs(np.pi * radius**2 - area)
    
    return np.array([area, area_c])

def get_segment_area(angles, radius):
    """Wrapper for cached segment area calculation"""
    if len(angles) == 0:
        return np.array([0.0, 0.0])
    
    # Convert angles to a string key for caching
    angles_key = ','.join([f"{angle:.6f}" for angle in angles])
    return get_segment_area_key(angles_key, radius)

def infoMetric(area1, area2):
    return 0.5 * np.abs(area1 + area2)

# Distance cache
distance_cache = {}

def get_distance(source, target):
    """Calculate distance with caching"""
    key = (source[0], source[1], source[2], target[0], target[1], target[2])
    if key not in distance_cache:
        distance_cache[key] = np.linalg.norm(np.array(source) - np.array(target))
    return distance_cache[key]

def info_map(arena, circle1_center, circle2_center, aperture, radius):
    x_resolution = 60
    y_resolution = 60

    x = np.linspace(0, arena.length, x_resolution)
    y = np.linspace(0, arena.width, y_resolution)

    info_mat = np.zeros((x_resolution, y_resolution))
    print(info_mat.shape, '\n')
    
    # Clear caches for new calculation
    distance_cache.clear()
    circle_points_cache.clear()
    
    # Convert to numpy arrays once for efficiency
    circle1 = np.array(circle1_center)
    circle2 = np.array(circle2_center)
    
    for i in tqdm(range(x_resolution)):
        for j in range(y_resolution):
            source = (x[i], y[j], 20)
            
            # Get visible angles
            visible_anglesL = get_visible_angles(source, circle1_center, radius, aperture)
            visible_anglesR = get_visible_angles(source, circle2_center, radius, aperture)

            # Calculate segment areas
            area_circle1 = get_segment_area(visible_anglesL, radius)
            area_circle2 = get_segment_area(visible_anglesR, radius)

            # Determine which area to use
            if len(visible_anglesL) > 180:
                A1 = np.max(area_circle1)
            else:
                A1 = np.min(area_circle1)
                
            if len(visible_anglesR) > 180:
                A2 = np.max(area_circle2)
            else:
                A2 = np.min(area_circle2)
                
            # Calculate distances (cached)
            d_left = get_distance(source, circle1_center)
            d_right = get_distance(source, circle2_center)

            # Calculate information metric
            info_mat[i, j] = infoMetric(A1/np.sqrt(d_left), A2/np.sqrt(d_right))
            
    return info_mat

# Main loop - unchanged
arena_length = 60
arena_width = 60
arena = Arena(length=arena_length, width=60, height=50)




In [ ]:
widths = np.arange(3, 15, 1)
for w in tqdm(widths):
    aperture = Aperture(arena_width=arena_width, arena_height=50, arena_length=arena_length, gap_width=w)
    circleL = (arena.width/2 - 10, arena.width, arena.height/2)
    circleR = (arena.width/2 + 10, arena.width, arena.height/2)
    info_mat = info_map(arena, circleL, circleR, aperture, radius=5)
    np.save(os.path.join(BASE_PATH, f'new_info_matrix_hres_{w}.npy'), info_mat)

In [ ]:
widths = np.arange(3, 15, 1)
for w in tqdm(widths):

    info_mat=np.load(os.path.join(BASE_PATH, f'new_info_matrix_hres_{w}.npy'))
    # info=np.flipud(info)
    plt.xlabel('world x')
    plt.ylabel('world y')
    plt.title(f'information gain at location, w={w}')
    c= plt.imshow(info_mat.T[:-10,],origin='lower', cmap='viridis')
    plt.colorbar(c, label='information gain at location')
    print(info_mat.shape)
    plt.show()